# The TypeformSync in debug mode

This notebook runs the TypeformSync module but lets you debug it.

Notebook and module written by Avi Alkalay &lt;avi at unix.sh&gt;

São Paulo, July 2019

In [1]:
from configobj import ConfigObj    # dnf install python3-configobj
import logging
import pandas as pd

config='syncFromTypeform.conf'

## Import the module

In [2]:
from Typeform import TypeformSync

## Initialize logger and sync module

In [3]:
context=ConfigObj(config)

# Switch between INFO/DEBUG while running in production/developping:
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger('Typeform').setLevel(logging.DEBUG)
logging.getLogger('urllib3.connectionpool').setLevel(logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler())


tf = TypeformSync(context['typeform_token'],context['database'])

## Sync

In [4]:
tf.sync()

Requesting form updates since 1970-01-01 00:00:00…
Requesting forms…
/usr/lib64/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
Requesting form items…
Requesting response statistics for form «Tz3iJK», submitted=true…
Requesting response statistics for form «Tz3iJK», submitted=false…
Requesting response statistics for form «sCBW64», submitted=true…
Requesting responses for form «sCBW64», submitted=true…
Requesting response statistics for form «sCBW64», submitted=false…
Requesting response statistics for form «deP0yD», submitted=true…
Requesting responses for form «deP0yD», submitted=true…
Requesting response statistics for form «deP0yD», submitted=false…
Requesting response statistics

## Some debugging starts here...

In [ ]:
logging.getLogger('Typeform').setLevel(logging.DEBUG)
# print(logging.getLogger('TypeForm').__dict__)

str(logging.getLogger('TypeForm').__dict__)

tf.logger.debug('abc')
logging.debug('123')

In [ ]:
logging.getLogger('Typeform').__dict__
tf.logger.debug(123)

In [ ]:
answers_db=pd.read_sql_table(con=tf.db,table_name='answers')

In [ ]:
answers_db.set_index('id', inplace=True)

In [ ]:
df = pd.merge(tf.answers, answers_db, how='outer', indicator=True)
rows_in_tf_not_in_db = df[df['_merge']=='left_only']
rows_in_db_not_in_tf = df[df['_merge']=='right_only']
tf.answers.sort_values(by='id').head(50)

In [ ]:
answers_db.sort_values(by='id').head(50)

In [ ]:
answers_db.shape

In [ ]:
# On a first sync, this should get you an empty DataFrame, meaning that DataFrame built from Typeform API and DB table are identical:
rows_in_tf_not_in_db

In [ ]:
# On a first sync, this should get you an empty DataFrame, meaning that DataFrame built from Typeform API and DB table are identical:
rows_in_db_not_in_tf

In [ ]:
tf.answers.data_type_hint.value_counts()

In [ ]:
tf.responses.sort_values('submitted')

In [ ]:
tf.answers['response']=pd.Categorical(tf.answers['response'],tf.responses.sort_values(by='landed').index)
tf.answers.sort_values(by='response')

In [ ]:
tf.formItems.type.value_counts()

In [ ]:
#11249
# tf.answers['answer']=tf.answers['safe_answer']
for i in tf.answers[26185:26189].index: #[11100:11105]:
    print(i,':')
    print(" ".join("{:04x}".format(ord(c)) for c in tf.answers.loc[i].answer))
    print()
# '\\xF0\\x9F\\x98\\x91'

In [ ]:
print('{:04x}'.format(ord('ç')))

In [ ]:
# tf.answers[25500+153:26249]
tf.answers[26185:26189].head(10)

In [ ]:
tf.answers.loc['1d0f26f04c'].answer

In [ ]:
tf.answers.loc[['0945c297c8']].answer.replace(to_replace=tf.invalidTextChars, value=None, inplace=True)

In [ ]:
tf.answers['answer']=tf.answers.answer.str.replace(u'\u200b', '')

In [ ]:
tf.answers.loc['2c99af2c35'].answer
#tf.answers

In [ ]:
print(u'\u200b')

In [ ]:
" ".join("{:02x}".format(ord(c)) for c in tf.answers.loc['0945c297c8'].answer)


In [ ]:
" ".join("{:02x}".format(ord(c)) for c in tf.answers.loc['0945c297c8'].answer.replace(u'\u200b',''))

In [ ]:
tf.answers.answer.replace(to_replace=tf.invalidTextChars, value=None, inplace=True)



In [ ]:
tf.answers2=pd.read_sql_table('answers',tf.db)

In [ ]:
tf.answers2.info()

In [ ]:
tf.answers.info()

In [ ]:
tf.answers2.set_index('id', inplace=True)

In [ ]:
tf.answers2['response']=pd.Categorical(tf.answers2['response'],tf.responses.sort_values(by='landed').index)
tf.answers2.sort_values(by='response', inplace=True)

In [ ]:
tf.answers2.head(100)

In [ ]:
tf.answers.head(100)

In [ ]:
df1=tf.answers
df2=tf.answers2

In [ ]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)

In [ ]:
df_2notin1 = df1[~(df1['id'].isin(df2['id']))].dropna().reset_index(drop=True)

In [ ]:
df_2notin1

In [ ]:
import numpy as np
df=pd.DataFrame(index=np.arange(100), columns=np.arange(2))

In [ ]:
df[0:3]

In [ ]:
df[3:4]

In [ ]:
import numpy as np
np.log(100000)